In [1]:
import Anti_HebbFF_Network as AHF
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import torchviz
import sys
import copy
import os
# from torch.utils.tensorboard import SummaryWriter

In [2]:
# meta parameters
vec_len = 100
T = 500
lr = 0.001
batch_size = 64
acc = 0.98
hid_dim = 100
out_dim = 1

In [3]:
HebbFF_CFD = AHF.HebbFF(input_dim = vec_len, hid_dim=hid_dim, out_dim=out_dim, batch_size = batch_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(HebbFF_CFD.parameters(),lr = lr)

## R = 1

In [ ]:
R = 1

In [ ]:
AHF.acc_train(HebbFF_CFD, acc, batch_size, vec_len, R, T, criterion, optimizer,print_log = 10)

In [ ]:
PATH = os.getcwd() + '/Model/R_{}'.format(R)
torch.save(HebbFF_CFD.state_dict(), PATH)

In [ ]:
ACC_R = []
for i in range(1, 151):
    input_seq,target_seq = AHF.generate_seq(batch_size = batch_size, vec_len = vec_len, R = i, T = T)
    accuracy,total_loss = AHF.test(HebbFF_CFD,criterion,optimizer,input_seq,target_seq,T,batch_size)
    ACC_R.append(accuracy.item())

r_axis = np.arange(1,151)
plt.plot(r_axis,ACC_R)
plt.title("Accuracy across different intervals after training with R in [1,150]")

In [ ]:
ACC_R = []
HebbFF_CFD.batch_size = 20
batch_size = 20
T = 1000
for i in range(1, 501):
    input_seq,target_seq = AHF.generate_seq(batch_size = batch_size, vec_len = vec_len, R = i, T = T)
    accuracy,total_loss = AHF.test(HebbFF_CFD,criterion,optimizer,input_seq,target_seq,T,batch_size)
    ACC_R.append(accuracy.item())

r_axis = np.arange(1,501)
plt.plot(r_axis,ACC_R)
plt.title("Accuracy across different intervals after training with R in [1,150]")

In [ ]:
HebbFF_CFD.batch_size = 1
HebbFF_CFD.A = torch.zeros(HebbFF_CFD.batch_size, 100,100)
batch_size = 1
T = 1000
R = 400
input_seq,target_seq = AHF.generate_seq(batch_size = batch_size, vec_len = vec_len, R = R, T = T)
out = []
for i in range(T):
    x_t = input_seq[:,i,:]
    target = target_seq[:,i,:]

    output = HebbFF_CFD(x_t)
    loss = criterion(output, target)
    total_loss += loss

    predicted = (output>0.5).float()
    out.append(predicted)
    accuracy  += (predicted == target).sum()


In [ ]:
batch_size = 64
HebbFF_CFD.batch_size = 64
HebbFF_CFD.A = torch.zeros(HebbFF_CFD.batch_size, 100,100)
T = 500
R = 1

In [ ]:
while True:
    R += 1
    print('R: ',R)
    AHF.acc_train(HebbFF_CFD, acc, batch_size, vec_len, R, T, criterion, optimizer,print_log = 100)
    PATH = os.getcwd() + '/Model/R_{}'.format(R)
    torch.save(HebbFF_CFD.state_dict(), PATH)